# Directive for creating a script for your notebook

The block here below is required at the top of each notebook that you want to create a script for. You will also need to edit the "settings.ini" file, to create a script (see [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details). Replace **some_string** with a name that makes sense for your notebook. 

In [ ]:
# |default_exp blast_parser


# Libraries
Include all the libraries which should be used in this module. You can also import modules from other notebooks; here, we have imported the functions in the core notebook.

In [ ]:
# |export

# standard libs
import os
import re

# Common to template
# add into settings.ini, requirements, package name is python-dotenv, for conda build ensure `conda config --add channels conda-forge`
import dotenv  # for loading config from .env files, https://pypi.org/project/python-dotenv/
import envyaml  # Allows to loads env vars into a yaml file, https://github.com/thesimj/envyaml
import fastcore  # To add functionality related to nbdev development, https://github.com/fastai/fastcore/
from fastcore import (
    test,
)
from fastcore.script import (
    call_parse,
)  # for @call_parse, https://fastcore.fast.ai/script
import json  # for nicely printing json and yaml

#import functions from core module (optional, but most likely needed). 
from ssi_analysis_result_parsers import(
    core
)

# Project specific libraries
from pathlib import Path
import pandas
import sys

In [ ]:
# This block should never be exported. It is to have python running in the project (and not the nbs) dir, and to initiate the package using pip.
os.chdir(core.PROJECT_DIR)

# Functions

Add your code here below. If your notebook will be used as a console-script, you need to add a "cli"-function, at the end (see [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Code execution** and **Input, output and options**) on loop for more details)

In [ ]:
# | export

def extract_presence_absence(blast_output_tsv: Path, 
                             tsv_header: str = "qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                             hits_as_string: bool = True,  
                             include_match_stats = False, 
                             pident_threshold: float = 90, 
                             plen_threshold: float = 60, 
                             gene_names: list = None
                             ) -> dict:
    """
    Parse blast output tsv for best matching alleles
    returns:
    if include_match stats:
        { <gene_name_1>: <allele_number>,  <gene_name_2>: <allele_number>,  <gene_name_3>: <allele_number> ...}
    else:
        a dictionary (allele_dict) in the format { <gene_name_1>: <allele_number>,  <gene_name_2>: <allele_number>,  <gene_name_3>: <allele_number> ...}

    """
    if os.path.exists(blast_output_tsv):
        try:
            blast_df = pandas.read_csv(blast_output_tsv, sep='\t', header = None)
            header_list = tsv_header.split(' ')
            if len(header_list) == len(blast_df.columns):
                blast_df.columns = tsv_header.split(' ')
                blast_df["plen"] = blast_df["length"]/blast_df["qlen"]*100
                blast_df_unique = blast_df.sort_values(by=['bitscore'], ascending= False).groupby("qseqid").first()
                blast_df_filtered = blast_df_unique.query(
                            "plen > @plen_threshold and pident > @pident_threshold"
                        )
                blast_dict = dict(blast_df_filtered.to_dict(orient = "index"))
            else:
                print(f"Failed to parse {blast_output_tsv}. Number of columns do not match length of provided header string", file=sys.stderr)
                return None
            
        except pandas.errors.EmptyDataError:
            blast_dict = {}
            print(f"Blast output file {blast_output_tsv} empty. Assuming 0 blast hits.")
        #except Exception as e:
        #    print(f"Error parsing blast: e")
        if hits_as_string:
            
            results = []
            for gene, d in blast_dict.items():
                if include_match_stats:
                    results.append(f"{gene}__{d['pident']}__{d['plen']}")
                else:
                    results.append(gene)
            result_dict = { "genes_found": ", ".join(results)}
            return result_dict

        else:
            result_dict = {}
            if gene_names is None:
                gene_names = list(blast_dict.keys())
            for gene in gene_names:
                if gene in blast_dict:
                    if include_match_stats:
                        result_dict[gene] = f"{blast_dict[gene]['pident']}__{blast_dict[gene]['plen']}"
                    else:
                        result_dict[gene] = "1"
                else:
                    result_dict[gene] = "0"
            return result_dict



    else:
        print(f"No blast output found at {blast_output_tsv}", file=sys.stderr)
        return None
    

def extract_allele_matches(blast_output_tsv: Path,
                           tsv_header: str,
                           include_match_stats = False) -> dict:
    """
    Parse blast output tsv for best matching alleles
    returns:
    if include_match stats:
        { <gene_name_1>: <allele_number>__<pident>__<plen>,  <gene_name_2>: <allele_number>__<pident>__<plen>,  <gene_name_3>: <allele_number>__<pident>__<plen> ...}
    else:
        a dictionary (allele_dict) in the format { <gene_name_1>: <allele_number>,  <gene_name_2>: <allele_number>,  <gene_name_3>: <allele_number> ...}

    """
    allele_dict = {}
    detailed_dict = {}
    if os.path.exists(blast_output_tsv):
        try:
            blast_df = pandas.read_csv(blast_output_tsv, sep='\t', header = None)
            header_list = tsv_header.split(' ')
            if len(header_list) == len(blast_df.columns):
                blast_df.columns = tsv_header.split(' ')
                blast_df.set_index("qseqid", drop=False)
                blast_df["plen"] = blast_df["length"]/blast_df["qlen"]*100
                blast_df[['gene','allele']] = blast_df['qseqid'].str.split('_',expand=True)
                blast_df_unique = blast_df.sort_values(by=['bitscore'], ascending= False).groupby("gene").first()
                for gene, d in blast_df_unique.to_dict(orient = "index").items():
                    allele_dict[gene] = d["allele"]
                    detailed_dict[gene] = f"{d['allele']}__{d['pident']}__{d['plen']}"
            else:
                print(f"Failed to parse {blast_output_tsv}. Number of columns do not match length of provided header string", file=sys.stderr)
                return None

        except pandas.errors.EmptyDataError:
            detailed_dict = {}
            allele_dict = {}
            print(f"Blast output file {blast_output_tsv} empty. Assuming 0 blast hits.")
    else:
        print(f"No blast output found at {blast_output_tsv}", file=sys.stderr)
        return None
    
    if include_match_stats:
        return detailed_dict
    else:
        return allele_dict

## TESTING


In [ ]:


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/blast_parser/gene_presence_absence_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=False,
                                     hits_as_string=True
                                     )
assert(len(gene_presence_dict) == 1)
assert(gene_presence_dict["genes_found"] == "asd, proA")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/blast_parser/gene_presence_absence_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True,
                                     hits_as_string=True,
                                     )
assert(gene_presence_dict["genes_found"] == "asd__98.732__100.0, proA__97.284__100.0")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/blast_parser/gene_presence_absence_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True,
                                     hits_as_string=True,
                                     pident_threshold=80,
                                     )
assert(gene_presence_dict["genes_found"] == "asd__98.732__100.0, pilE__82.835__100.0, proA__97.284__100.0")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/blast_parser/gene_presence_absence_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=False,
                                     hits_as_string=False,
                                     gene_names = ["asd","proA","pilE"])
assert(gene_presence_dict["asd"] == "1")
assert(gene_presence_dict["proA"] == "1")
assert(gene_presence_dict["pilE"] == "0")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/blast_parser/gene_presence_absence_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True,
                                     hits_as_string=False,
                                     gene_names = ["asd","proA","pilE"])

assert(gene_presence_dict["asd"] == "98.732__100.0")
assert(gene_presence_dict["proA"] == "97.284__100.0")
assert(gene_presence_dict["pilE"] == "0")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/blast_parser/empty_gene_presence_absense_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=False,
                                     hits_as_string=False,
                                     gene_names=["lag-1"]
                                     )
assert(gene_presence_dict["lag-1"] == "0")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/empty_file.txt",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=False,
                                     hits_as_string=False,
                                     gene_names=["lag-1"]
                                     )
assert(gene_presence_dict["lag-1"] == "0")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/empty_file.txt",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True,
                                     hits_as_string=False,
                                     gene_names=["lag-1"]
                                     )
assert(gene_presence_dict["lag-1"] == "0")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/empty_file.txt",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=False,
                                     hits_as_string=True,
                                     gene_names=["lag-1"]
                                     )
assert(gene_presence_dict["genes_found"] == "")


gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/empty_file.txt",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True,
                                     hits_as_string=True
                                     )
assert(gene_presence_dict["genes_found"] == "")


# Test for missing file handling (should return None)
gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/file/that/does/not/exist.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True,
                                     hits_as_string=True,
                                     gene_names=["lag-1"]
                                     )
assert(gene_presence_dict is None)

# Check for incorrect number of columns handling (should return None)
gene_presence_dict = extract_presence_absence(blast_output_tsv="./test_input/Legionella/test.sbt.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True,
                                     hits_as_string=True,
                                     gene_names=["lag-1"]
                                     )
assert(gene_presence_dict is None)


allele_dict = extract_allele_matches(blast_output_tsv="./test_input/blast_parser/allele_matches_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore")

detailed_allele_dict = extract_allele_matches(blast_output_tsv="./test_input/blast_parser/allele_matches_test.tsv",
                                     tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                     include_match_stats=True)

assert(len(allele_dict) == 7)
assert(allele_dict['asd'] == "9")
assert(detailed_allele_dict['pilE'] == "3__100.0__100.0")


# Test for missing file handling (should return None)
allele_dict = extract_allele_matches(blast_output_tsv="./test_input/file/that/does/not/exist.tsv",
                                            tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                            include_match_stats=True)
assert(allele_dict is None)


# Test for incorrect number of columns handling (should return None)
allele_dict = extract_allele_matches(blast_output_tsv="./test_input/Legionella/test.sbt.tsv",
                                            tsv_header="qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",
                                            include_match_stats=True)
assert(allele_dict is None)

Blast output file ./test_input/blast_parser/empty_gene_presence_absense_test.tsv empty. Assuming 0 blast hits.
Blast output file ./test_input/empty_file.txt empty. Assuming 0 blast hits.
Blast output file ./test_input/empty_file.txt empty. Assuming 0 blast hits.
Blast output file ./test_input/empty_file.txt empty. Assuming 0 blast hits.
Blast output file ./test_input/empty_file.txt empty. Assuming 0 blast hits.


No blast output found at ./test_input/file/that/does/not/exist.tsv
Failed to parse ./test_input/Legionella/test.sbt.tsv. Number of columns do not match length of provided header string
No blast output found at ./test_input/file/that/does/not/exist.tsv
Failed to parse ./test_input/Legionella/test.sbt.tsv. Number of columns do not match length of provided header string


In [ ]:
# |export
from fastcore.script import call_parse


@call_parse
def presence_absence(
    blast_output: Path = None,  # Path to blast output file. Generated with --outfmt 6 option
    blast_tsv_header: str = "qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",  # headers in blast output
    hits_as_string: bool = True,  # True to print a comma separated list of found genes on a single line. False to return a key: value pair for each gene
    include_match_stats: bool = False, # True to include percent identity and percent length in output, false to only include present/absent
    percent_identityt: float = 90,  # percent identity threshold for considering a gene present
    percent_length: float = 60,  # percent length threshold for considering a gene present
    gene_names: list = None,  # name of genes to look for when hits_as_string = False
    output_file: Path = None,
    config_file: str = None,  # config file to set env vars from
) -> None:
    """
    
    """
    #config = core.get_config(config_file)  # Set env vars and get config variables
    gene_presence_dict = extract_presence_absence(blast_output_tsv=blast_output,
                                                  tsv_header=blast_tsv_header,
                                                  hits_as_string=hits_as_string,
                                                  include_match_stats=include_match_stats,
                                                  pident_threshold=percent_identityt,
                                                  plen_threshold=percent_length,
                                                  gene_names=gene_names,
                                                  )
    
    


@call_parse
def allele_matches(
    blast_output: Path = None,  # Path to blast output file. Generated with --outfmt 6 option
    blast_tsv_header: str = "qseqid sseqid pident length qlen qstart qend sstart send sseq evalue bitscore",  # headers in blast output
    include_match_stats: bool = False, # True to include percent identity and percent length in output, false to only include allele number
    output_file: Path = None,
    config_file: str = None,  # config file to set env vars from
) -> None:
    """
    
    """
    #config = core.get_config(config_file)  # Set env vars and get config variables
    allele_dict = extract_allele_matches(blast_output_tsv=blast_output,
                                         tsv_header=blast_tsv_header,
                                         include_match_stats=include_match_stats,
                                         output_file=None,
                                         )
    

# Directive for ensuring that the code in your notebook get executed as a script

The code-block here below is required to ensure that the code in the notebook is also transferred to the module (script), otherwise it will just be a notebook. See [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details.

In [ ]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the module and isn't just a notebook
import nbdev

nbdev.nbdev_export()